In [1]:
# импортируем необходимые модули
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np 
import faiss

import tqdm

h:\RUTUBE_HACK\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# считываем тестовые данные
data = pd.read_csv("./sample_submission_5.csv")
data

Unnamed: 0                          video_id  \
0             0  c00d1b173ed8680c155b773d43af7791   
1             1  6064c4511a067f2e1d1cef81c21421e8   
2             2  3071e2054a02a03150463e514f05fe90   
3             3  60867179613376c8b56f0705114a7685   
4             4  706f44ef4711eaee99035260c2d1ab86   
..          ...                               ...   
194         194  0f84219f46782fca9482fb88da146ed3   
195         195  2f67917a65dd8d5b348de81d15b1d8ff   
196         196  6fa39a883591bc0e2cf7ef6c5349c9df   
197         197  bff493ac045715566a86cd7e84051938   
198         198  7fdc692721faaaa5cd74f99b361cf593   

                                                 title  \
0    Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. З...   
1    БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | К...   
2    Такой расклад. Эфир 17. Таро. Про личные качес...   
3    TREXTRAVELER в ГрандТуре «Байкальская миля 202...   
4                                    Тру ДЕТЕКТОР I #5   
..                                                 ...   
194           Диктант со звездой | Выпуск 7 | Чипинкос   
195                            Полупановы. «Эпизод 16»   
196                 Салют, Начальник! 2 сезон, 6 серия   
197        Абьюз-шоу |Выпуск №7 часть 1I Алина Егорова   
198  Хашлама | Выпуск 15 I twinsrussian – парень за...   

                                           description  \
0    Утро Тяпы начинается с недвусмысленного напоми...   
1    Добро пожаловать на канал “Бюро Добрых Дел”! Е...   
2    В этом выпуске шоу «Такой расклад» вы увидите ...   
3    Визит в Государственный природный заказник "Су...   
4    Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...   
..                                                 ...   
194  В этом выпуске - рэпер Чипинкос останется один...   
195  В этом эпизоде Игорь готовится к любимому праз...   
196  Назаров устраивает майору «сюрприз» прямо за с...   
197  Героиней этого выпуска «Абьюз шоу» стала Алина...   
198  Сеты в Якитории – это подвид греха уныние.  Бл...   

                                        predicted_tags  
0    ['Массовая культура', 'Массовая культура: Юмор...  
1    ['Массовая культура: Юмор и сатира', 'Массовая...  
2                 ['Религия и духовность: Астрология']  
3    ['Путешествия', 'События и достопримечательнос...  
4    ['Массовая культура', 'Массовая культура: Юмор...  
..                                                 ...  
194  ['Массовая культура', 'Массовая культура: Юмор...  
195                              ['Массовая культура']  
196  ['Массовая культура: Юмор и сатира', 'Массовая...  
197  ['Семья и отношения', 'Личные финансы', 'Карье...  
198  ['Семья и отношения', 'Массовая культура: Юмор...  

[199 rows x 5 columns]

In [4]:
# загружаем обученную модель
model_path = "./winstrike_rutube_tags_model" # путь до папки с моделью SentenceTransformer
model = SentenceTransformer(model_path, )
dim = 768 # размер вектора эмбеддинга

In [5]:
# Чтение embedding тегов
with open('./tags_embeddings.npy', "rb") as f:
    np_tags_arr = np.load(f)

In [6]:
# Чтение всех тегов
with open('./organized_tags.txt', "r", encoding="utf-8") as f:
    all_tags_list = f.read().split('\n')

In [7]:
def get_predict_from_title_and_description(title : str, description : str) -> list:
    """ Получение тегов к видео по наименованию и описанию """
    global model, dim, np_tags_arr
    
    embedding = model.encode(title + " " + description, convert_to_tensor=True).cpu().numpy()
    index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
    index.add(np_tags_arr)

    topn = 3
    scores, predictions = index.search(np.array([list(embedding)], dtype=np.float64), topn)

    predictions = [all_tags_list[idx] for idx in predictions[0]]
    final_tags = []

    for idx, pred in enumerate(predictions):
        is_skeep = False
        for idx_2, pred_2 in enumerate(predictions):
            if (pred in pred_2) and (idx_2 != idx):
                is_skeep = True
            
        if is_skeep:
            continue
        else:
            final_tags.append(pred)


    return scores[0], final_tags

In [8]:
# переносим данные во второй DataFrame для преобразования
copied_data = data.copy()
copied_data

Unnamed: 0                          video_id  \
0             0  c00d1b173ed8680c155b773d43af7791   
1             1  6064c4511a067f2e1d1cef81c21421e8   
2             2  3071e2054a02a03150463e514f05fe90   
3             3  60867179613376c8b56f0705114a7685   
4             4  706f44ef4711eaee99035260c2d1ab86   
..          ...                               ...   
194         194  0f84219f46782fca9482fb88da146ed3   
195         195  2f67917a65dd8d5b348de81d15b1d8ff   
196         196  6fa39a883591bc0e2cf7ef6c5349c9df   
197         197  bff493ac045715566a86cd7e84051938   
198         198  7fdc692721faaaa5cd74f99b361cf593   

                                                 title  \
0    Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. З...   
1    БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | К...   
2    Такой расклад. Эфир 17. Таро. Про личные качес...   
3    TREXTRAVELER в ГрандТуре «Байкальская миля 202...   
4                                    Тру ДЕТЕКТОР I #5   
..                                                 ...   
194           Диктант со звездой | Выпуск 7 | Чипинкос   
195                            Полупановы. «Эпизод 16»   
196                 Салют, Начальник! 2 сезон, 6 серия   
197        Абьюз-шоу |Выпуск №7 часть 1I Алина Егорова   
198  Хашлама | Выпуск 15 I twinsrussian – парень за...   

                                           description  \
0    Утро Тяпы начинается с недвусмысленного напоми...   
1    Добро пожаловать на канал “Бюро Добрых Дел”! Е...   
2    В этом выпуске шоу «Такой расклад» вы увидите ...   
3    Визит в Государственный природный заказник "Су...   
4    Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...   
..                                                 ...   
194  В этом выпуске - рэпер Чипинкос останется один...   
195  В этом эпизоде Игорь готовится к любимому праз...   
196  Назаров устраивает майору «сюрприз» прямо за с...   
197  Героиней этого выпуска «Абьюз шоу» стала Алина...   
198  Сеты в Якитории – это подвид греха уныние.  Бл...   

                                        predicted_tags  
0    ['Массовая культура', 'Массовая культура: Юмор...  
1    ['Массовая культура: Юмор и сатира', 'Массовая...  
2                 ['Религия и духовность: Астрология']  
3    ['Путешествия', 'События и достопримечательнос...  
4    ['Массовая культура', 'Массовая культура: Юмор...  
..                                                 ...  
194  ['Массовая культура', 'Массовая культура: Юмор...  
195                              ['Массовая культура']  
196  ['Массовая культура: Юмор и сатира', 'Массовая...  
197  ['Семья и отношения', 'Личные финансы', 'Карье...  
198  ['Семья и отношения', 'Массовая культура: Юмор...  

[199 rows x 5 columns]

In [10]:
# наименование колонки, содержащей данные, подаваемые на вход модели
BASED_COL = "title_plus_description"

In [11]:
# склеиваем строки
copied_data[BASED_COL] = copied_data["title"] + " " + copied_data["description"]

In [12]:
# преобразуем из series -> list
test_data_list = copied_data[BASED_COL].to_list()
test_data_list

['Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. ЗВЕЗДА В ШОКЕ Утро Тяпы начинается с недвусмысленного напоминания Лизы о подарке ей на 8 Марта, про который он, разумеется, забыл, но признаваться в этом не стал. Пообещав ей, что сюрприз будет готов к вечеру, Тяпа обращается за помощью в выборе подарка к своему другу Чернобылю. Как он ему в этом поможет и понравится ли сюрприз Лизе, а также, что Стас Михайлов забыл под ее балконом  – вы узнаете, посмотрев эту серию.',
 'БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | КОЛБАСНАЯ ПЛАНЕТА | АКЦИЯ «БЕСПЛАТНОЕ ТЕПЛО» Добро пожаловать на канал “Бюро Добрых Дел”! Его глава – это я, Лариса Брохман. А миссия нашего Бюро – искать самые добрые и хорошие новости по всему миру, чтобы рассказать их вам. В этом выпуске:  - Как обычный рыбак спасает морское дно? - Почему кусочек колбаски мировые СМИ приняли за планету? - Ксения Егорова в рамках рубрики «Добрый репортаж» провела акцию «Бесплатное тепло» и подарила москвичам немного уюта этой осенью. - Михаил 

In [11]:
# ВЫВОД ТЕГОВ БЕЗ СОХРАНЕНИЯ В ФАЙЛ

# for (idx, cur_title_and_desc) in enumerate(tqdm.tqdm(test_data_list)):
#     scores, predicted_tags = get_predict_from_title_and_description(copied_data["title"][idx], copied_data["description"][idx])
#     print(f"[\'{"\', \'".join(predicted_tags)}\']")

In [13]:
# предикт тегов, занесение их в DataFrame
for (idx, cur_title_and_desc) in enumerate(tqdm.tqdm(test_data_list)):
    scores, predicted_tags = get_predict_from_title_and_description(copied_data["title"][idx], copied_data["description"][idx])
    data["predicted_tags"][idx] = f"[\'{"\', \'".join(predicted_tags)}\']"

  0%|          | 0/199 [00:00<?, ?it/s]C:\Users\bruse\AppData\Local\Temp\ipykernel_32052\597233942.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["predicted_tags"][idx] = f"[\'{"\', \'".join(predicted_tags)}\']"
C:\Users\bruse\AppD

In [14]:
# сохранение результата в *.csv
data.to_csv("./result.csv")